# 🔍 Hệ Thống Nghiên Cứu Pháp Luật Việt Nam (v2.0)

**Cập nhật mới:**
- Tách biệt **Retriever Agent** (Tra cứu nội bộ RAG) và **Searcher Agent** (Tra cứu Web).
- Hỗ trợ kiểm tra hiệu lực văn bản theo thời gian thực.
- Bắt buộc trích dẫn chi tiết (Điều, Khoản, Điểm).

## Hướng dẫn sử dụng:
1. Chạy cell **Setup** để khởi tạo hệ thống
2. **Chọn cơ sở dữ liệu RAGFlow**
3. Nhập câu hỏi pháp lý
4. Chạy cell **Chạy Nghiên Cứu**

⚠️ **Lưu ý:** Vui lòng **Restart Kernel** trước khi chạy nếu bạn vừa cập nhật code.

---
## 1️⃣ Setup - Khởi tạo hệ thống

In [ ]:
import sys
import os
import asyncio
import logging
import requests

# Add src to path
project_root = os.path.dirname(os.path.abspath('.'))
if project_root not in sys.path:
    sys.path.insert(0, os.getcwd())

# Set logging level
logging.basicConfig(level=logging.WARNING, format='%(asctime)s - %(levelname)s - %(message)s')

# Load environment variables
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

# Import AutoGen components
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.ui import Console

# Import our modules
from src.config import get_model_client
from src.agents.planner import create_planner_agent
from src.agents.retriever import create_retriever_agent
from src.agents.searcher import create_searcher_agent
from src.agents.analyzer import create_analyzer_agent
from src.agents.critic import create_critic_agent
from src.agents.synthesizer import create_synthesizer_agent
from src.tools import set_knowledge_ids, get_knowledge_ids

print("✅ Đã import thành công tất cả modules!")

# Initialize model client
try:
    model_client = get_model_client()
    print("✅ Đã kết nối với AI model!")
except Exception as e:
    print(f"❌ Lỗi kết nối model: {e}")
    model_client = None

# Create agents
if model_client:
    planner = create_planner_agent(model_client)
    retriever = create_retriever_agent(model_client)
    searcher = create_searcher_agent(model_client)
    analyzer = create_analyzer_agent(model_client)
    critic = create_critic_agent(model_client)
    synthesizer = create_synthesizer_agent(model_client)
    print("✅ Đã khởi tạo tất cả agents: Planner, Retriever, Searcher, Analyzer, Critic, Synthesizer!")
    print("\n🎯 Hệ thống sẵn sàng! Tiếp tục chọn cơ sở dữ liệu.")

---
## 2️⃣ Chọn Cơ Sở Dữ Liệu RAGFlow

Chạy cell dưới đây để chọn dataset.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def get_ragflow_datasets():
    """Fetch list of available datasets from RAGFlow."""
    base_url = os.getenv('RAGFLOW_BASE_URL', 'http://localhost:9380')
    api_key = os.getenv('RAGFLOW_API_KEY')
    try:
        url = f"{base_url}/api/v1/datasets"
        headers = {'Authorization': f'Bearer {api_key}'}
        response = requests.get(url, headers=headers, timeout=5)
        data = response.json()
        if data.get('code') == 0:
            return data.get('data', [])
        return []
    except:
        return []

datasets = get_ragflow_datasets()
print(f"📚 Tìm thấy {len(datasets)} datasets:\n")
options = []
for d in datasets:
    label = f"{d.get('name')} ({d.get('document_count')} docs)"
    print(f"  • {label}")
    options.append((label, d.get('id')))

if options:
    dropdown = widgets.Dropdown(options=options, description='Dataset:')
    btn = widgets.Button(description='✓ Xác nhận', button_style='success')
    out = widgets.Output()
    def on_click(b):
        with out:
            clear_output()
            set_knowledge_ids([dropdown.value])
            print(f"✅ Đã chọn ID: {dropdown.value}")
    btn.on_click(on_click)
    display(widgets.HBox([dropdown, btn]), out)
else:
    print("⚠️ Không tìm thấy datasets hoặc lỗi kết nối.")

---
## 3️⃣ Nhập câu hỏi pháp lý

In [ ]:
# ========================================
# 📝 NHẬP CÂU HỎI CỦA BẠN TẠI ĐÂY:
# ========================================

query = "Quy định tách dự án đầu tư"
print(f"📋 Câu hỏi: {query}")

---
## 4️⃣ Chạy Nghiên Cứu (Update Multi-Agent)

Hệ thống sẽ chạy với quy trình:
1. **Planner**: Lập kế hoạch.
2. **Retriever**: Lấy văn bản từ RAG.
3. **Searcher**: Check hiệu lực online.
4. **Analyzer**: Phân tích & Trích dẫn.
5. **Critic**: Kiểm duyệt.
6. **Synthesizer**: Tổng hợp báo cáo.

In [ ]:
def extract_research_context(result):
    """Extract text content from research result to pass to synthesizer."""
    context_parts = []
    if not result or not hasattr(result, 'messages'): return ""
    for msg in result.messages:
        source = getattr(msg, 'source', 'Unknown')
        content = getattr(msg, 'content', None)
        if content:
            if isinstance(content, str):
                context_parts.append(f"=== {source} ===\n{content}")
            elif isinstance(content, list):
                for part in content:
                    if hasattr(part, 'content'):
                        context_parts.append(f"=== {source} (Tool Result) ===\n{str(part.content)[:3000]}")
    return "\n\n".join(context_parts)

async def run_legal_research(query: str):
    print("="*60)
    print("🚀 BẮT ĐẦU NGHIÊN CỨU PHÁP LUẬT (MULTI-AGENT)")
    print("="*60)
    
    # === PHASE 1: Research & Critique ===
    print("\n📚 GIAI ĐOẠN 1: NGHIÊN CỨU VÀ ĐÁNH GIÁ")
    print("Agents: Planner → Retriever → Searcher → Analyzer → Critic")
    print("-"*60)
    
    research_termination = TextMentionTermination("APPROVE") | MaxMessageTermination(max_messages=15)
    
    # Added 'searcher' to the team
    research_team = RoundRobinGroupChat(
        participants=[planner, retriever, searcher, analyzer, critic],
        termination_condition=research_termination,
    )
    
    task = f"Legal Query: {query}\nPlease plan, retrieve internal docs, verify validity online, and analyze."
    research_result = await Console(research_team.run_stream(task=task))
    
    # === PHASE 2: Synthesis ===
    print("\n📝 GIAI ĐOẠN 2: TỔNG HỢP BÁO CÁO")
    research_context = extract_research_context(research_result)
    
    synthesis_team = RoundRobinGroupChat(
        participants=[synthesizer],
        termination_condition=MaxMessageTermination(max_messages=2),
    )
    
    synthesis_task = f"""Bạn được giao nhiệm vụ tổng hợp báo cáo pháp lý từ kết quả nghiên cứu dưới đây.

=== CÂU HỎI ===
{query}

=== KẾT QUẢ NGHIÊN CỨU ===
{research_context}

=== YÊU CẦU ===
Tạo báo cáo pháp lý chi tiết với:
1. Tóm tắt vấn đề
2. Cơ sở pháp lý (Trích dẫn cụ thể Điều, Khoản, Điểm)
3. Phân tích hiệu lực (Dựa trên check của Searcher)
4. Kết luận

Viết báo cáo ngay."""
    
    synthesis_result = await Console(synthesis_team.run_stream(task=synthesis_task))
    print("\n✅ HOÀN THÀNH")
    return research_result, synthesis_result

# Run
if model_client:
    research_result, synthesis_result = await run_legal_research(query)

---
## 📄 Xuất Báo Cáo Cuối Cùng

In [ ]:
from IPython.display import display, Markdown
if 'synthesis_result' in dir() and hasattr(synthesis_result, 'messages'):
    for msg in reversed(synthesis_result.messages):
        if 'Synthesizer' in getattr(msg, 'source', ''):
            display(Markdown(getattr(msg, 'content', '')))
            break